In [61]:
# Code from Chapter 17 of Machine Learning: An Algorithmic Perspective (2nd Edition)
# by Stephen Marsland (http://stephenmonika.net)

# You are free to use, change, or redistribute the code in any way you wish for
# non-commercial purposes, but please maintain the name of the original author.
# This code comes with no warranty of any kind.

# Stephen Marsland, 2014

import numpy as np
#import pylab as pl

class ds_rbm:
    def __init__(self,nvisible,nhidden,nlabels=None,inputs=[],
                 eta=0.1,momentum=0.9,decay=0.,nCDsteps=5,nepochs=1000):
        self.nvisible = nvisible
        self.nhidden = nhidden
        
        self.weights = np.random.randn(nvisible,nhidden)*0.1
        self.visiblebias = np.random.randn(nvisible)*0.1
        self.hiddenbias = np.random.randn(nhidden)*0.1
        
        self.nlabels = nlabels
        
        if nlabels is not None:
            self.labelweights = np.random.randn(nlabels,nhidden)*0.1
            self.labelbias = np.random.randn(nlabels)*0.1

        # cf Hinton 8.1
        if np.shape(inputs)[0]>0:
            p = np.sum(inputs,axis=1)/(np.shape(inputs)[1])
            self.visiblebias = np.log(p/(1-p))

        self.eta = eta
        self.momentum = momentum
        self.decay = decay
        self.nCDsteps = nCDsteps
        self.nepochs = nepochs

    # Sample the visible variables
    def compute_visible(self,hidden):
        # Compute p(v=1|h,W), p(l=1|h,W)
        sumin = self.visiblebias + np.dot(hidden,self.weights.T)

        # Compute visible node activations
        self.visibleprob = 1./(1. + np.exp(-sumin))
        self.visibleact = (
            self.visibleprob>np.random.rand(
                np.shape(self.visibleprob)[0],self.nvisible
            )).astype('float')

        # Compute label activations (softmax)
        if self.nlabels is not None:
            sumin = self.labelbias + np.dot(hidden,self.labelweights.T)
            summax = sumin.max(axis=1)
            summax = np.reshape(summax,summax.shape+(1,)).repeat(np.shape(sumin)[1],axis=-1)            
            sumin -= summax
            normalisers = np.exp(sumin).sum(axis=1)
            normalisers = np.reshape(normalisers,normalisers.shape+(1,)).repeat(np.shape(sumin)[1],axis=-1)            
            self.labelact = np.exp(sumin)/normalisers

        return [self.visibleprob,self.visibleact,self.labelact]

    # Sample the hidden variables
    def compute_hidden(self,visible,label=[]):
        
        # Compute p(h=1|v,W,{l})
        if self.nlabels is not None:
            sumin = self.hiddenbias + np.dot(visible,self.weights) + np.dot(label,self.labelweights)
        else:
            sumin = self.hiddenbias + np.dot(visible,self.weights)
        self.hiddenprob = 1./(1. + np.exp(-sumin))
        
        # num of data x num of hidden
        self.hiddenact = (
            self.hiddenprob>np.random.rand(
                np.shape(self.hiddenprob)[0],self.nhidden
            )).astype('float')
        return [self.hiddenact,self.hiddenprob]

    def classify(self,inputs,labels):
        h, ph = self.compute_hidden(inputs,labels)
        #h, ph = self.compute_hidden(inputs,1./(labels.max()+1)*np.ones(np.shape(labels)))
        v, pv, l = self.compute_visible(h)
        
        print ( l.argmax(axis=1) )
        print (labels.argmax(axis=1))
        
        print ('Errors:', (l.argmax(axis=1) != labels.argmax(axis=1)).sum())               

    def energy(self, visible, hidden, labels=[]):
        if self.nlabels is not None:
            ret_en = (
                -np.dot(visible, self.visiblebias) 
                -np.dot(hidden , self.hiddenbias) 
                -np.dot(labels , self.labelbias) 
                - (np.dot(visible, self.weights     )*hidden).sum(axis=1) 
                - (np.dot(labels , self.labelweights)*hidden).sum(axis=1)
            )
            return ret_en
        else:
            
            # (num_data x num_dimension) x (num_dimension x num_hidden) 
            # * (num_data x num_hidden)
            ret_en = (
                -np.dot(visible, self.visiblebias) 
                -np.dot(hidden , self.hiddenbias) 
                -(np.dot(visible, self.weights)*hidden).sum(axis=1)
            )
            # dsaint31's debug
            # print(visible.shape)
            # print(self.weights.shape)
            # print(hidden.shape)
            return ret_en 

    def contrastive_divergence(self,inputs,labels=None,
                               dw=None,dwl=None,dwvb=None,
                               dwhb=None,dwlb=None,silent=False):

        # Clamp input into visible nodes
        visible = inputs
        
        #self.visibleact = inputs
        self.labelact = labels
        
        dw = 0. if dw is None else dw
        
        dwl = 0. if dwl is None else dwl
        dwvb = 0. if dwvb is None else dwvb
        dwhb = 0. if dwhb is None else dwhb
        dwlb = 0. if dwlb is None else dwlb
        
        for epoch in range(self.nepochs):
            
            # dsaint31 debug
            #print(epoch,"[visible_bias:",self.visiblebias,"|hidden_bias:",self.hiddenbias,"]")
            
            # Sample the hidden variables
            self.compute_hidden(visible,labels)
    
            # Compute <vh>_0
            positive = np.dot(inputs.T,self.hiddenact)            
            #positive = np.dot(inputs.T,self.hiddenprob)
            
            # Contrastive Divergence : Positive_Visible_Bias
            positive_vb = inputs.sum(axis=0)
            
            # Contrastive Divergence : Positive_Hidden_Bias
            positive_hb = self.hiddenprob.sum(axis=0)
            
            if self.nlabels is not None:
                positive_labels = np.dot(labels.T,self.hiddenact)
                #positive_labels = np.dot(labels.T,self.hiddenprob)
                positivelb = labels.sum(axis=0)

            ###################################
            # Asleep phase
            # Do limited Gibbs sampling to sample from the hidden distribution
            for j in range(self.nCDsteps):    
                self.compute_visible(self.hiddenact)
                #print("labelact:"+str(self.labelact))
                self.compute_hidden(self.visibleact,self.labelact)

            # Compute <vh>_n
            negative = np.dot(self.visibleact.T,self.hiddenact)
            #negative = np.dot(self.visibleact.T,self.hiddenprob)
            
            # Contrastive Divergence : Negative_Visible_Bias
            negative_vb = self.visibleact.sum(axis=0)
            
            # Contrastive Divergence : Negative_Hidden_Bias
            negative_hb = self.hiddenprob.sum(axis=0)

            if self.nlabels is not None:
                negative_labels = np.dot(self.labelact.T,self.hiddenact)
                #negative_labels = np.dot(self.labelact.T,self.hiddenprob)
                negative_lb = self.labelact.sum(axis=0)
                dwl = self.eta * ((positive_labels - negative_labels) / np.shape(inputs)[0] - self.decay*self.labelweights) + self.momentum*dwl
                self.labelweights += dwl
                dwlb = self.eta * (positivelb - negative_lb) / np.shape(inputs)[0] + self.momentum*dwlb
                self.labelbias += dwlb

            ###########################################
            # Learning rule (with momentum)
            # - weights
            # - visiblebias
            # - hiddenbias
            dw = self.eta * (
                (positive - negative) / np.shape(inputs)[0] 
                - self.decay*self.weights) + self.momentum*dw
            self.weights += dw

            dwvb = self.eta * ((positive_vb - negative_vb) / np.shape(inputs)[0] 
                              )+ self.momentum*dwvb
            self.visiblebias += dwvb
            
            dwhb = self.eta * ((positive_hb - negative_hb) / np.shape(inputs)[0]
                              ) + self.momentum*dwhb
            self.hiddenbias += dwhb

            # error = sum of square
            error = np.sum((inputs - self.visibleact)**2)
            
            if (epoch%50==0) and not silent: 
                print (epoch, error/np.shape(inputs)[0], self.energy(visible,self.hiddenprob,labels).sum())

            # for Awake    
            visible = inputs
            self.labelact = labels

        if not silent:
            self.compute_hidden(inputs,labels)
            #print ("energy:",self.energy(visible,self.hiddenprob,labels).sum())
            print ("energy:",self.energy(visible,self.hiddenprob,labels))
        return error 


    def cddemo(self,inputs,labels=None,dw=None,dwl=None,dwvb=None,dwhb=None,dwlb=None,silent=False):

        # Clamp input into visible nodes
        visible = inputs
        #self.visibleact = inputs
        self.labelact = labels
        
        dw = 0. if dw is None else dw
        dwl = 0. if dwl is None else dwl
        dwvb = 0. if dwvb is None else dwvb
        dwhb = 0. if dwhb is None else dwhb
        dwlb = 0. if dwlb is None else dwlb
        
        for epoch in range(self.nepochs):
            print (epoch)
            # Sample the hidden variables
            print (self.compute_hidden(visible,labels))
    
            # Compute <vh>_0
            positive = np.dot(inputs.T,self.hiddenact)
            #positive = np.dot(inputs.T,self.hiddenprob)
            positive_vb = inputs.sum(axis=0)
            positivehb = self.hiddenprob.sum(axis=0)

            print (positive, positive_vb, positivehb)

            # Do limited Gibbs sampling to sample from the hidden distribution
            for j in range(self.nCDsteps):    
                print (self.compute_visible(self.hiddenact))
                print (self.compute_hidden(self.visibleact,self.labelact))

            # Compute <vh>_n
            negative = np.dot(self.visibleact.T,self.hiddenact)
            #negative = np.dot(self.visibleact.T,self.hiddenprob)
            negativevb = self.visibleact.sum(axis=0)
            negativehb = self.hiddenprob.sum(axis=0)
            print (negative, negativevb, negativehb)

            # Learning rule (with momentum)
            dw = self.eta * ((positive - negative) / np.shape(inputs)[0] - self.decay*self.weights) + self.momentum*dw
            self.weights += dw

            dwvb = self.eta * (positive_vb - negativevb) / np.shape(inputs)[0] + self.momentum*dwvb
            self.visiblebias += dwvb
            dwhb = self.eta * (positivehb - negativehb) / np.shape(inputs)[0] + self.momentum*dwhb
            self.hiddenbias += dwhb

            error = np.sum((inputs - self.visibleact)**2)
            if (epoch%50==0) and not silent: 
                print (epoch, error/np.shape(inputs)[0], self.energy(visible,self.hiddenprob,labels).sum())

            visible = inputs
            self.labelact = labels

        if not silent:
            self.compute_hidden(inputs,labels)
            print (self.energy(visible,self.hiddenprob,labels).sum())
        return error 

In [62]:
# nvisible,nhidden,nlabels=None,inputs=[],eta=0.1,momentum=0.9,decay=0.,nCDsteps=5,nepochs=1000):

def test_rbm3():
    
    #import rbm
    # visible = 5
    # hidden = 2
    r = ds_rbm(5, 2,nCDsteps=1,momentum=0.9,nepochs = 100)
    inputs = np.array(
        [[0,1,0,1,1],[1,1,0,1,0],[1,1,0,0,1],[1,1,0,0,1], [1,0,1,0,1],[1,1,1,0,0]]
    )
    r.contrastive_divergence(inputs)
    
    #r.cddemo(inputs)
    print ("weights")
    print (r.weights)
    print ("visiblebias")
    print (r.visiblebias)
    print ("hiddenbias")
    print (r.hiddenbias)
    print ("hiddenact :shape["+str(r.hiddenact.shape)+"]")
    print (r.hiddenact)
    print ("---")
    print ()
    
    # recall
    test = np.array([[1,1,0,0,1]])
    r.compute_hidden(test)
    print (r.hiddenact)
    print (r.hiddenact.shape)

    test = np.array([[1,0]])
    r.compute_visible(test)
    print (r.visibleact)

    test = np.array([[1,0]])
    r.compute_visible(test)
    print (r.visibleact)

    test = np.array([[1,0]])
    r.compute_visible(test)
    print (r.visibleact)

In [63]:
test_rbm3()

0 3.0 -1.64324107641
50 1.33333333333 -22.3809132335
energy: [-6.24388596 -5.60775218 -8.28837841 -8.28837841 -6.31150147 -6.91637083]
weights
[[-1.44476295  3.71527859]
 [ 3.42942419  0.53455128]
 [-4.82306848  1.9164637 ]
 [ 1.78137862 -1.80077105]
 [ 1.09949549  0.39772464]]
visiblebias
[ 0.96602661  1.08846625 -0.37432778 -0.49342551  0.55176798]
hiddenbias
[-0.92524516 -0.8194087 ]
hiddenact :shape[(6, 2)]
[[ 1.  0.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 0.  1.]
 [ 0.  1.]]
---

[[ 0.  1.]]
(1, 2)
[[ 0.  1.  0.  0.  1.]]
[[ 1.  1.  0.  1.  1.]]
[[ 0.  1.  0.  1.  1.]]


In [16]:
import numpy as np

v = [1,0,0]
W= np.eye(3)
np.dot(v,W)


array([ 1.,  0.,  0.])

In [26]:
inputs = np.array(
        [[0,1,0,1,1],[1,1,0,1,0],[1,1,0,0,1],[1,1,0,0,1], [1,0,1,0,1],[1,1,1,0,0]]
    )
inputs.sum(axis=0)

array([5, 5, 2, 2, 4])

In [38]:
t = 

k = 0 if t == 3 else 5
print(k)

5
